 # Trending Topics: Daily Digest for Central Banks Announcements



 ## Automated Analysis of Central Bank Communications and Market Impact



 ## Why It Matters



 Central bank announcements are among the most market-moving events in global finance, capable of triggering significant volatility across currency, bond, and equity markets within minutes. A single Fed chair comment or ECB policy shift can move trillions in asset values, making timely identification of emerging monetary policy narratives critical for success. However, manually tracking the flood of central bank communications, policy speeches, and market reactions across multiple time zones is both resource-intensive and prone to delays that can cost traders and analysts critical market positioning opportunities.



 ## What It Does



 Using the functions available in the `bigdata-research-tools` package, this workflow creates a comprehensive central bank communications monitoring system that automatically identifies, verifies, and ranks trending topics from monetary policy announcements. Built for traders, analysts, portfolio managers, and policy researchers, it transforms scattered central bank news into structured daily intelligence reports with quantified market impact scores, enabling rapid response to policy shifts and enhanced decision-making in volatile market conditions.



 ## How It Works



 This workflow combines:



 - **Specialized Lexicon Generation** creating monetary policy and central bank-specific terminology to maximize recall in news retrieval and capture nuanced policy language



 - **Multi-threaded Content Retrieval** via the Bigdata API, splitting searches into daily windows and parallelizing keyword lookups for comprehensive speed and coverage



 - **AI-powered Topic Clustering & Verification** to filter noise, group related news into coherent themes, and summarize key narratives with scoring for trendiness, novelty, impact direction, and magnitude


to deliver **customizable daily market digests** with **configurable ranking** systems, allowing users to prioritize by volume, novelty, or market impact, supported by granular news sources for verification



 ## A Real-World Use Case


 This cookbook demonstrates the complete workflow through analyzing central bank communications during the week of the ECB Forum in Sintra (June 30 - July 6, 2025), a period when high-profile speeches from global central bankers including Christine Lagarde, Jerome Powell, and other key policymakers coincided with Donald Trump's remarks on Federal Reserve monetary policy. You'll learn how to transform unstructured policy-related news into structured, ranked insights that reveal market-moving announcements before they fully impact asset prices.

 ## Setup and Imports

 ## Async Compatibility Setup



 **Run this cell first** - Required for Google Colab, Jupyter Notebooks, and VS Code with Jupyter extension:



 ### Why is this needed?



 Interactive environments (Colab, Jupyter) already have an asyncio event loop running. When bigdata-research-tools makes async API calls (like to OpenAI), you'll get this error without nest_asyncio:



 ```

 RuntimeError: asyncio.run() cannot be called from a running event loop

 ```



 The `nest_asyncio.apply()` command patches this to allow nested event loops.



 💡 **Tip**: If you're unsure which environment you're in, just run the cell below - it won't hurt in any environment!

In [1]:
import datetime
start = datetime.datetime.now()

try:
    import asyncio
    asyncio.get_running_loop()
    import nest_asyncio; nest_asyncio.apply()
    print("✅ nest_asyncio applied")
except (RuntimeError, ImportError):
    print("✅ nest_asyncio not needed or not available")


✅ nest_asyncio applied


 ## Environment Setup



 The following cell configures the necessary path for the analysis

In [2]:
import os
import sys


current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)
print(f"✅ Local environment setup complete")


✅ Local environment setup complete


 ## Import Required Libraries



 Import the core libraries needed for trending topics analysis, including the lexicon generation classes and data processing tools.

In [3]:
from bigdata_client import Bigdata
from bigdata_client.models.search import DocumentType
from src.lexicon_generator import LexiconGenerator
from src.search_topics import search_by_keywords
from src.topics_extractor import (process_all_reports,
                                run_process_all_trending_topics,
                                run_add_advanced_novelty_scores,
                                add_market_impact_to_df,
                                prepare_data_for_report,
                                generate_html_report)
from IPython.display import display
from IPython.core.display import HTML


 ## Define Output Paths



 Define the output paths for trending topics analysis results.

In [4]:
# Define output file paths for our report
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

export_path = f"{output_dir}/trending_topics_central_banks_results.xlsx"


 ## Load Credentials

In [5]:
from dotenv import load_dotenv
from pathlib import Path

script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
#load_dotenv(script_dir / '.env')
load_dotenv(os.path.abspath("/home/abouchs/.python_env_var/.env"))

BIGDATA_USERNAME = os.getenv('BIGDATA_USERNAME')
BIGDATA_PASSWORD = os.getenv('BIGDATA_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not all([BIGDATA_USERNAME, BIGDATA_PASSWORD, OPENAI_API_KEY]):
    print("❌ Missing required environment variables")
    raise ValueError("Missing required environment variables. Check your .env file.")
else:
    print("✅ Credentials loaded from .env file")


✅ Credentials loaded from .env file


 ## Connecting to Bigdata



 Create a Bigdata object with your credentials.

In [6]:
bigdata = Bigdata(BIGDATA_USERNAME, BIGDATA_PASSWORD)


 ## Defining Your Trending Topics Analysis Parameters





 ### Core Parameters

 -  **Main Theme** (`main_theme`): The central concept to explore for trending topics analysis (e.g., Central Bank Announcements, Monetary Policy, Interest Rate Decisions)

 -  **Time Period** (`start_date` and `end_date`): The date range over which to run the analysis. For optimal results, use windows of 1-2 weeks for trending topics analysis

 -  **Document Type** (`document_type`): Specify which documents to search over (transcripts, filings, news). News is recommended for trending topics due to timely coverage

 -  **Model Selection** (`llm_model`): The LLM model used for semantic analysis, topic clustering, and impact scoring


 ### Customizable Parameters

 -  **Frequency** (`freq`): The frequency of the date ranges to search over. Should match your reporting frequency.

    Supported values:



    -  `Y`: Yearly intervals.

    -  `M`: Monthly intervals.

    -  `W`: Weekly intervals.

    -  `D`: Daily intervals. Defaults to `D` for trending topics.



 -  **Document Limit** (`document_limit`): The maximum number of documents to return per query to Bigdata API. Higher limits provide more comprehensive coverage but increase processing time



 -  **Semaphore Size** (`semaphore_size`): Controls the maximum number of concurrent requests to OpenAI API for processing efficiency



 -  **Batches** (`batches`): Number of batches to process the reports in parallel (adjust based on your system's capabilities and rate limits)

In [7]:
# ===== Context Definition =====
main_theme = "Central Bank Announcements"

# ===== Specify Time Range =====
start_query = '2025-06-30'
end_query = '2025-07-06'

# ===== Query Configuration =====
document_type = DocumentType.NEWS

# ===== LLM Specification =====
llm_model = "gpt-4o-mini"


 ## Instantiate the Lexicon Generator

In [8]:
LexiconGenerator = LexiconGenerator(openai_key=OPENAI_API_KEY, model="gpt-4o", seeds=[123, 123456, 123456789, 456789, 789])

In [9]:
keywords = LexiconGenerator.generate(theme=main_theme)

 ## Retrieve Content using Bigdata's Search Capabilities



 With the generated keywords, you can leverage the multithreaded Search functionalities in `bigdata-research-tools` to run search at scale against news documents.

In [10]:
# ===== Query Configuration =====
document_limit = 10 # Maximum number of documents to retrieve per query
frequency = 'D'  # Query frequency

In [11]:
results, daily_keyword_count = search_by_keywords(
    keywords=keywords,
    start_date=start_query,
    end_date=end_query,
    freq=frequency,
    scope=document_type,
    document_limit=document_limit)


About to run 364 queries
Example Query: Keyword('Monetary Policy') over date range: AbsoluteDateRange('2025-06-30T00:00:00', '2025-06-30T23:59:59')


Querying Bigdata...: 100%|██████████| 364/364 [01:16<00:00,  4.78it/s]


 ## Topic Clustering and Summarization



 Apply a verification layer to remove the news that are not relevant to our `main_theme`.

In [12]:
semaphore_size = 1000 # Maximum number of concurrent requests to Openai API
 
# Filter reports to remove news not related to Central Bank announcements
filtered_reports = process_all_reports(results, llm_model, OPENAI_API_KEY, main_theme, semaphore_size)


Filtering News:   0%|          | 0/4947 [00:00<?, ?it/s]

Perform topic clustering using a large language model to label and cluster the news. Then, summarize each topic and generate a daily summary, identifying the top trending news for Central Banks Announcements.

In [13]:
# Perform topic modeling and clustering to identify key trending topics
flattened_trending_topics_df = run_process_all_trending_topics(
    unique_reports=filtered_reports,
    model=llm_model,
    start_query=start_query,
    end_query=end_query,
    api_key=OPENAI_API_KEY,
    main_theme=main_theme,
    batches=20 #number of batches to process the reports in parallel (adjust based on your system's capabilities)
)


Extracting Topics for 2025-06-30:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-07-01:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-07-02:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-07-03:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-07-04:   0%|          | 0/19 [00:00<?, ?it/s]

...

Consolidating topic batches: 100%|██████████| 8/8 [00:20<00:00,  2.55s/it]


Summarizing text for each topic...


Generating titles: 100%|██████████| 38/38 [00:06<00:00,  5.99it/s]


Generating Day in Review summaries...
Adding one-line summaries to DataFrame...


Generating text summaries: 100%|██████████| 1517/1517 [01:03<00:00, 23.84it/s] 


 ## Topic Scoring

 **Trendiness and Novelty Scores**: Derive analytics related to the trendiness of the topic based on the news volume, and the novelty of the topic based on the changes in daily summaries, evaluating the uniqueness and freshness of each topic.

In [14]:
# Calculate trendiness and novelty scores, assessing the uniqueness and freshness of each topic
flattened_trending_topics_df = run_add_advanced_novelty_scores(flattened_trending_topics_df, api_key = os.environ['OPENAI_API_KEY'], main_theme = main_theme)


Calculating Novelty Scores:   0%|          | 0/74 [00:00<?, ?it/s]

 **Price Impact**: Derive analytics related to the impact (Positive, Negative) and magnitude (High, Medium, Low) of the topics. To customize the analysis, set an additional parameter:
 
-  **Point of View** (`point_of_view`): The context and additional instructions for a better interpretation of the trending topics and their likely impact.

The price impact inference is based on the price mechanisms and the perceived sentiment and market reaction of the news on the market.. Setting `point_of_view` to `Domestic Equity Market`  will lead the LLM to infer the topics' market impact on equity prices. 

In [15]:
# Assess the market impact for each topic, evaluating how each topic influences Equity market
point_of_view = 'Domestic Equity market'
flattened_trending_topics_df = add_market_impact_to_df(flattened_trending_topics_df, api_key = os.environ['OPENAI_API_KEY'], main_theme = main_theme, point_of_view = point_of_view)


 For verification purpose, this actionable timestamped dataframe contains the granular news clustered into relevant topics, and also the advanced analytics of trendiness, novelty, impact, and magnitude scores to be used for backtesting.

In [16]:
flattened_trending_topics_df.head()

,Date,Day_in_Review,Topic,Summary,Source,Headline,Text,Volume_Score,Text_Summary,Volume_Score,Novelty_Score,Impact_Score,Magnitude_Score
0,2025-06-30,- **PBoC's Policy**: Implementing a cautiously...,PBoC's Cautiously Accommodative Policy Aims to...,The People's Bank of China (PBoC) is implement...,Central Banking,PBoC to increase 'intensity' of policy adjustm...,"Bank to retain ""moderately loose"" stance while...",2,The People's Bank of China plans to maintain a...,2,Moderate,Positive,Medium
25,2025-06-30,- **PBoC's Policy**: Implementing a cautiously...,Divergent Central Bank Policies Drive Global I...,Interest rate expectations and decisions are c...,Yahoo! Finance,House sales keep increasing but price inflatio...,Interest rates are expected to continue to fal...,6,The Bank of England's cautious stance amid hig...,6,New,Negative,High
3,2025-06-30,- **PBoC's Policy**: Implementing a cautiously...,Divergent Central Bank Policies Drive Global I...,Interest rate expectations and decisions are c...,ABC Online,Stocks climbed a 'wall of worry' to gain 10pc ...,Any major oil price surge had the potential to...,2,A significant rise in oil prices could trigger...,2,New,Negative,High
4,2025-06-30,- **PBoC's Policy**: Implementing a cautiously...,Bank of England Cuts Rates to Boost Housing Ma...,The Bank of England (BoE) is currently navigat...,ShareCast via Web,Mortgage approvals push higher in May - BoE,The BoE has trimmed the cost of borrowing twic...,5,The Bank of England has reduced borrowing cost...,5,New,Positive,Medium
5,2025-06-30,- **PBoC's Policy**: Implementing a cautiously...,Bank of England Cuts Rates to Boost Housing Ma...,The Bank of England (BoE) is currently navigat...,Money Age,Monthly net mortgage approvals rise for first ...,Net mortgage approvals for house purchases inc...,5,"In May, net mortgage approvals rose to 63,000,...",5,New,Positive,Medium


 ## Generate a Custom Daily Digest

 Finally, produce a customizable report in the form of a Daily Digest, displaying the top trending topics and their summaries according to your custom ranking. 
 
 The following parameters are needed to prepare the data for the report:

 - **Specific Date** (`specific_date`): The selected date for the report.

 - **User Selected Ranking** (`user_selected_ranking`): The list of features to prioritize when ranking, such as `volume` (trendiness and media attention), `novelty` (based on the emergence of new daily news), `impact` direction (positive or negative) and `magnitude`.
 
 - **Impact Filter** (`impact_filter`): Whether to filter the report for only positive or negative news. Default `None`.

 The ranking system prioritizes the criteria in the order specified, allowing for a tailored focus on the most relevant aspects of the data.


In [17]:
specific_date = '2025-07-01'  # Example date, can be modified as needed
user_selected_ranking = ['novelty', 'volume', 'magnitude']  # User can modify this list to change the ranking order
#impact_filter = 'positive_impact' #User can use the impact_filter to filter out the report


In [18]:
prepared_reports = prepare_data_for_report(flattened_trending_topics_df, user_selected_ranking, impact_filter = None, report_date = specific_date)


Finally display a daily market update.

In [19]:
# Generate and display the HTML report for each date
for report in prepared_reports:
    html_content = generate_html_report(
        report['date'],
        report['day_in_review'],
        report['topics'],
        main_theme  # Pass the main theme to dynamically generate the title
    )
    display(HTML(html_content))
    print("")
    print("")
    print("")


## Key Insights and Analysis
 The trending topics analysis reveals distinct patterns in central bank communications and market impact:

**Political Pressure Dominance**: The Fed-RBA topic emerges as the most significant story with 30 news articles, reflecting intense political scrutiny on Federal Reserve independence amid Trump's criticism of Powell's rate decisions. This represents the highest volume topic, indicating sustained market attention on central bank autonomy.

**Geographic Policy Divergence**: Analysis reveals a clear split between developed and emerging market approaches, with the Fed, ECB, and BoJ facing restrictive policy challenges while emerging central banks (RBI, CBN) pursue more accommodative stances to support growth, highlighting divergent global monetary policy cycles.

**Novel vs. Recurring Narrative Mix**: 5 topics scored as "Novel" (33%), 3 as "Moderate" (20%), and 7 as "Repeat" (47%), suggesting that while half the coverage focuses on ongoing themes like inflation management, a significant portion represents genuinely new policy developments requiring immediate attention.

**Negative Impact Bias**: 10 out of 15 topics (67%) show negative market impact, with only 4 positive impacts, indicating that central bank communications during this period primarily reflected economic challenges and policy uncertainties rather than growth-supportive narratives.

 ## Export the Results



 Export the data as Excel files for further analysis or to share with the team.

In [ ]:
try:
    from bigdata_research_tools.excel import ExcelManager
    
    # Create the Excel manager
    excel_manager = ExcelManager()

    # Define the dataframes and their sheet configurations
    df_args = [
        (flattened_trending_topics_df, "Trending Topics Analysis", (0, 0)),
    ]

    # Save the workbook
    excel_manager.save_workbook(df_args, export_path)
    print(f"✅ Results exported to {export_path}")

except Exception as e:
    print(f"Warning while exporting to excel: {e}")
    # Fallback to CSV export
    flattened_trending_topics_df.to_csv(f"{output_dir}/trending_topics_central_banks_analysis.csv", index=False)
    print(f"✅ Results exported to CSV as fallback")


In [ ]:
print("Runtime:", datetime.datetime.now() - start)
